In [ ]:
import random
import pandas as pd
random.seed(42)

attendance_raw = []
cohorts = ["alpha", "beta", "gama"]

for i in range(1, 25):

    record = {
        "student_id": f"S{i:03}",
        "cohort": random.choice(cohorts),
        "attended_sessions": random.randint(0, 6),
        "expected_sessions": 6
    }

    attendance_raw.append(record)

df_attendance = pd.DataFrame(attendance_raw)

print(df_attendance.iloc[:5])


In [ ]:
attendance_indexed = df_attendance.set_index("student_id")

students_with_missing = attendance_indexed[
    attendance_indexed["attended_sessions"] < 6
].index.tolist()

total_excused = 10
num_extra = 3
num_existing = total_excused - num_extra

existing_ids = random.sample(students_with_missing, min(num_existing, len(students_with_missing)))

existing_values = [
    random.randint(1, min(3, attendance_indexed.loc[sid, "expected_sessions"] - attendance_indexed.loc[sid, "attended_sessions"]))
    for sid in existing_ids
]

extra_ids = ["S025", "S026", "S027"]  # exactly num_extra
extra_values = [2, 3, 1]

excused_absences = pd.concat([
    pd.Series(existing_values, index=existing_ids),
    pd.Series(extra_values, index=extra_ids)
])

attendance_indexed["adjusted_attendance"] = (
    attendance_indexed["attended_sessions"] + excused_absences
)

attendance_indexed["adjusted_attendance"] = attendance_indexed["adjusted_attendance"].fillna(
    attendance_indexed["attended_sessions"]
)

print(attendance_indexed[["attended_sessions", "adjusted_attendance"]])





In [ ]:
random_indices = random.sample(list(attendance_indexed.index), 5)

for idx in random_indices:

    value = attendance_indexed.loc[idx, "cohort"]
    value = f" {value.upper()} " if random.random() < 0.5 else f"{value.lower()}  "
    attendance_indexed.loc[idx, "cohort"] = value

attendance_indexed["cohort"] = attendance_indexed["cohort"].str.strip().str.lower()

print(attendance_indexed["cohort"].unique())

In [ ]:
low_attendance = attendance_indexed[
    attendance_indexed["attended_sessions"] < attendance_indexed["expected_sessions"]
]

avg_attendance_by_cohort = low_attendance.groupby("cohort")["attended_sessions"].mean()
print(avg_attendance_by_cohort)

print(low_attendance["cohort"].unique())
print(avg_attendance_by_cohort.index.tolist())

In [ ]:
attendance_indexed["attendance_ok"] = attendance_indexed["attended_sessions"] >= attendance_indexed["expected_sessions"]
low_attendance = attendance_indexed[attendance_indexed["attended_sessions"] < attendance_indexed["expected_sessions"]]


(low_attendance["attendance_ok"] == False).all()